In [44]:
# 输入数据的参数
_INPUT1='{"name":"input1","type":0,"uri":"tmp_be7bcb091e7c4ac4908d22ad0e5fea3c"}'
_INPUT2='{"name":"input2","type":0,"uri":"tmp_8366cfda4a2e4c7299ad76bd6b7dea88"}'
_INPUT3='{"name":"input3","type":0,"uri":"tmp_c88b437bd7c142e2b69d607adf50e28a"}'
_INPUT4='{"name":"input4","type":0,"uri":"tmp_b2f76de551d94edfa6d3c4dae75612c6"}'

# 输出数据的参数
_OUTPUT='[{"name":"output1","type":0,"uri":"tmp_731598a71f59446d815e84480ed484b3"},{"name":"output2","type":0,"uri":"tmp_f9338c98758945a6bc1df3c1bd2a047c"},{"name":"output3","type":0,"uri":"tmp_15ee0ada57f84c9e8ec312a956593af1"},{"name":"output4","type":0,"uri":"tmp_27499c3211b0441199059a1b50801718"}]'

# 自定义参数


In [45]:
!pip install lightgbm==2.2.3
!pip install xgboost

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [111]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
import gc
import wfio
import os
from collections import Counter
from ustciscrBDL_B import get_score_B
from ustciscrLab_A import get_score
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['font.sans-serif']=['SimHei']

In [112]:
def get_data(input):
    data = wfio.read_dataframe(input)
    return data

In [113]:
train = get_data(_INPUT1)
train2 = get_data(_INPUT2)
test = get_data(_INPUT3)
testb = get_data(_INPUT4)

train.to_csv('train.csv', index=False)
train2.to_csv('train2.csv', index=False)
test.to_csv('test.csv', index=False)
testb.to_csv('testb.csv', index=False)

In [114]:
train = pd.read_csv('train.csv')
train2 = pd.read_csv('train2.csv')
test = pd.read_csv('test.csv')
testb = pd.read_csv('testb.csv')
test['target'] = -1
testb['target'] = -1
train_len = len(train)
train2_len = len(train2)
test_len = len(test)
testb_len = len(testb)
before = set(train2.columns)

In [115]:
def get_2map():
    cols_chinese = ['2017Q1发票核定数量', '2017Q1进项金额', '2017Q1进项税率', '2017Q1开票金额', '2017Q1开票数', '2017Q1开票税额', '2017Q1开票税率', '2017Q1入库税额', 
    '2017Q2发票核定数量', '2017Q2进项金额', '2017Q2进项税率', '2017Q2开票金额', '2017Q2开票数', '2017Q2开票税额', '2017Q2开票税率', '2017Q2入库税额', 
    '2017Q3发票核定数量', '2017Q3进项金额', '2017Q3进项税率', '2017Q3开票金额', '2017Q3开票数', '2017Q3开票税额', '2017Q3开票税率', '2017Q3入库税额', 
    '2017Q4发票核定数量', '2017Q4进项金额', '2017Q4进项税率', '2017Q4开票金额', '2017Q4开票数', '2017Q4开票税额', '2017Q4开票税率', '2017Q4入库税额', 
    '2018Q1发票核定数量', '2018Q1进项金额', '2018Q1进项税率', '2018Q1开票金额', '2018Q1开票数', '2018Q1开票税额', '2018Q1开票税率', '2018Q1入库税额', 
    '2018Q2发票核定数量', '2018Q2进项金额', '2018Q2进项税率', '2018Q2开票金额', '2018Q2开票数', '2018Q2开票税额', '2018Q2开票税率', '2018Q2入库税额', 
    '2018Q3发票核定数量', '2018Q3进项金额', '2018Q3进项税率', '2018Q3开票金额', '2018Q3开票数', '2018Q3开票税额', '2018Q3开票税率', '2018Q3入库税额', 
    '2018Q4发票核定数量', '2018Q4进项金额', '2018Q4进项税率', '2018Q4开票金额', '2018Q4开票数', '2018Q4开票税额', '2018Q4开票税率', '2018Q4入库税额', 
    '2019Q1发票核定数量', '2019Q1进项金额', '2019Q1进项税率', '2019Q1开票金额', '2019Q1开票数', '2019Q1开票税额', '2019Q1开票税率', '2019Q1入库税额', 
    '2019Q2发票核定数量', '2019Q2进项金额', '2019Q2进项税率', '2019Q2开票金额', '2019Q2开票数', '2019Q2开票税额', '2019Q2开票税率', '2019Q2入库税额', 
    '2019Q3发票核定数量', '2019Q3进项金额', '2019Q3进项税率', '2019Q3开票金额', '2019Q3开票数', '2019Q3开票税额', '2019Q3开票税率', '2019Q3入库税额', 
    '2019Q4发票核定数量', '2019Q4进项金额', '2019Q4进项税率', '2019Q4开票金额', '2019Q4开票数', '2019Q4开票税额', '2019Q4开票税率', '2019Q4入库税额', 
    '财务人员手机号', '财务人员姓名', '从业人数', '登记开业日期', '法定代表人姓名', '法人手机号', '街道乡镇', '经营范围', '纳税人名称', '生产经营地址', 
    '行业', '行业大类', '行业门类', '行业中类', '注册地址', '注册资本（元）', '主键:纳税人识别号', 'target']
    cols_pin = ['2017q1fphdsl', '2017q1jxje', '2017q1jxsl', '2017q1kpje', '2017q1kps', '2017q1kpse', '2017q1kpsl', '2017q1rkse', 
    '2017q2fphdsl', '2017q2jxje', '2017q2jxsl', '2017q2kpje', '2017q2kps', '2017q2kpse', '2017q2kpsl', '2017q2rkse', 
    '2017q3fphdsl', '2017q3jxje', '2017q3jxsl', '2017q3kpje', '2017q3kps', '2017q3kpse', '2017q3kpsl', '2017q3rkse',
    '2017q4fphdsl', '2017q4jxje', '2017q4jxsl', '2017q4kpje', '2017q4kps', '2017q4kpse', '2017q4kpsl', '2017q4rkse', 
    '2018q1fphdsl', '2018q1jxje', '2018q1jxsl', '2018q1kpje', '2018q1kps', '2018q1kpse', '2018q1kpsl', '2018q1rkse', 
    '2018q2fphdsl', '2018q2jxje', '2018q2jxsl', '2018q2kpje', '2018q2kps', '2018q2kpse', '2018q2kpsl', '2018q2rkse', 
    '2018q3fphdsl', '2018q3jxje', '2018q3jxsl', '2018q3kpje', '2018q3kps', '2018q3kpse', '2018q3kpsl', '2018q3rkse', 
    '2018q4fphdsl', '2018q4jxje', '2018q4jxsl', '2018q4kpje', '2018q4kps', '2018q4kpse', '2018q4kpsl', '2018q4rkse', 
    '2019q1fphdsl', '2019q1jxje', '2019q1jxsl', '2019q1kpje', '2019q1kps', '2019q1kpse', '2019q1kpsl', '2019q1rkse', 
    '2019q2fphdsl', '2019q2jxje', '2019q2jxsl', '2019q2kpje', '2019q2kps', '2019q2kpse', '2019q2kpsl', '2019q2rkse', 
    '2019q3fphdsl', '2019q3jxje', '2019q3jxsl', '2019q3kpje', '2019q3kps', '2019q3kpse', '2019q3kpsl', '2019q3rkse', 
    '2019q4fphdsl', '2019q4jxje', '2019q4jxsl', '2019q4kpje', '2019q4kps', '2019q4kpse', '2019q4kpsl', '2019q4rkse', 
    'bsrxmmp', 'bsrxm', 'cyrs', 'djrq', 'fddbrxm', 'fdbrxmp', 'xzjd', 'jyfw', 'nsrmc', 'scjydz', 
    'hy', 'hydl', 'hyml', 'hyzl', 'zcdz', 'zczb', 'zjnsrsbh', 'yc']
    #
    chinese_pin = {i:j for i,j in zip(cols_chinese, cols_pin)}
    pin_chinese = {j:i for i,j in zip(cols_chinese, cols_pin)}
    return chinese_pin, pin_chinese

chinese_pin, pin_chinese = get_2map()

train = train.rename(columns=pin_chinese)
overlap = before & set(train.columns)
print(overlap)
assert(len(overlap)==0)

train2 = train2.rename(columns=pin_chinese)
overlap = before & set(train2.columns)
print(overlap)
assert(len(overlap)==0)

test = test.rename(columns=pin_chinese)
overlap = before & set(test.columns)
print(overlap)
assert(len(overlap)==0)

testb = testb.rename(columns=pin_chinese)
overlap = before & set(testb.columns)
print(overlap)
assert(len(overlap)==0)


stool = {'cwrysjh':'财务人员手机号', 'cwryxm':'财务人员姓名', 'djkyrq':'登记开业日期', 'frsjh':'法人手机号',
    'jdxz':'街道乡镇', 'zczby':'注册资本（元）', 'xy':'行业', 'xydl':'行业大类',
    'xyml':'行业门类','xyzl':'行业中类'}

train = train.rename(columns=stool)


cols_order = train.columns
train2 = train2.loc[:, cols_order]
test = test.loc[:, cols_order]
testb = testb.loc[:, cols_order]

set()
set()
set()
set()


In [116]:
main_col = [col for col in test.columns if col[0] != '2']
num_col = [col for col in test.columns if col[0] == '2']
num_col_date = list(set([i[:6] for i in num_col]))
num_col_type = list(set([i[6:] for i in num_col]))
num_col_date.sort()


In [117]:
def convert_2int(string):
    try:
        return float(string)
    except:
        return np.nan


def preprocess_main(df, is_test=False):
    if is_test: 
        df['2018Q3开票数'] = df['2018Q3开票数'].apply(convert_2int)
        
    df['main_na_count'] = df[main_col].isna().sum(axis=1)
    df['num_na_count'] = df[num_col].isna().sum(axis=1)
    df['财务人员姓名'] = df['财务人员姓名'].fillna('None')
    df['从业人数'] = df['从业人数'].fillna(-1)
    df['经营范围'] = df['经营范围'].fillna('None')
    df['生产经营地址'] = df['生产经营地址'].fillna('None')
    df['街道乡镇'] = df['街道乡镇'].fillna('None')
    df['注册地址'] = df['注册地址'].fillna('None')
    df['注册资本（元）'] = df['注册资本（元）'] = -1   # 2.000000e+06, -1
    return df

train = preprocess_main(train)
train2 = preprocess_main(train2)
test = preprocess_main(test, True)
testb = preprocess_main(testb)

In [118]:
duplicate = set(train2['主键:纳税人识别号']) & set(test['主键:纳税人识别号'])
print(len(duplicate))
duplicate = set(test['主键:纳税人识别号']) & set(testb['主键:纳税人识别号'])
len(duplicate)

3031


1768

In [119]:
train2.dtypes.value_counts()

float64    89
object     15
int64      12
dtype: int64

In [120]:
testb.dtypes.value_counts()

float64    89
object     15
int64      12
dtype: int64

In [121]:
def separate_main(data):
    # 人名重复
    data['财务人员重复'] = data.groupby('财务人员姓名')['财务人员姓名'].transform("count") 
    data['法定代表人重复'] = data.groupby('法定代表人姓名')['法定代表人姓名'].transform("count")
    name = list(data['法定代表人姓名']) +  list(data['财务人员姓名'])
    name_cnt = Counter(name)
    data['人员重复'] = data['财务人员姓名'].map(name_cnt) + data['法定代表人姓名'].map(name_cnt)
    print(len(name_cnt))

    # 手机重复
    phone = list(data['财务人员手机号']) +  list(data['法人手机号'])
    phone_cnt = Counter(phone)
    data['财务人员手机号重复'] = [phone_cnt[i] for i in data['财务人员手机号']]
    data['法人手机号重复'] = [phone_cnt[i] for i in data['法人手机号']]
    print(len(phone_cnt))
    return data

In [122]:
def concat(df1, df2, df3=None):
    df1 = separate_main(df1)
    df2 = separate_main(df2)
    cols_order = df1.columns
    df2 = df2.loc[:, cols_order]
    df1 = df1.append(df2)
    if df3 is not None:
        df3 = separate_main(df3)
        df3 = df3.loc[:, cols_order]
        df1 = df1.append(df3)
    print(df1.shape)
    return df1

train = concat(train, test)

13712
19980
7387
5646
(16000, 121)


In [123]:
def main_extract(data):
    # 日期
    data['登记开业日期'] = pd.to_datetime(data['登记开业日期']) 
    main_col.append('登记开业_year')
    main_col.append('登记开业_month')
    data['登记开业_year'] = data['登记开业日期'].dt.year
    data['登记开业_month'] = data['登记开业日期'].dt.month

    # 人名重复
#     data['财务人员重复'] = data.groupby('财务人员姓名')['财务人员姓名'].transform("count") 
#     data['法定代表人重复'] = data.groupby('法定代表人姓名')['法定代表人姓名'].transform("count")
#     name = list(data['法定代表人姓名']) +  list(data['财务人员姓名'])
#     name_cnt = Counter(name)
#     data['人员重复'] = data['财务人员姓名'].map(name_cnt) + data['法定代表人姓名'].map(name_cnt)
#     print(len(name_cnt))

#     # 手机重复
#     phone = list(data['财务人员手机号']) +  list(data['法人手机号'])
#     phone_cnt = Counter(phone)
#     data['财务人员手机号重复'] = [phone_cnt[i] for i in data['财务人员手机号']]
#     data['法人手机号重复'] = [phone_cnt[i] for i in data['法人手机号']]
#     print(len(phone_cnt))

    # 行业+地址
    data['行业_count'] = data.groupby('行业')['行业'].transform('count')
    data['行业大类_count'] = data.groupby('行业大类')['行业大类'].transform('count')
    data['行业门类_count'] = data.groupby('行业门类')['行业门类'].transform('count')
    data['行业中类_count'] = data.groupby('行业中类')['行业中类'].transform('count')
    data['街道乡镇_count'] = data.groupby("街道乡镇")['街道乡镇'].transform('count')

    data['address_match'] = (data['注册地址'] == data['生产经营地址'])*1
    address = list(data['生产经营地址']) +  list(data['注册地址'])
    address_cnt = Counter(address)
    data['生产经营地址_count'] = data.groupby('生产经营地址')['生产经营地址'].transform('count')
    data['注册地址_count'] = data.groupby('注册地址')['注册地址'].transform('count')
    data['地址重复'] = data['注册地址'].map(address_cnt) + data['生产经营地址'].map(address_cnt)

    # 行业编码
    str2int = LabelEncoder()
    data['行业'] = str2int.fit_transform(data['行业'])
    data['行业大类'] = str2int.fit_transform(data['行业大类'])
    data['行业门类'] = str2int.fit_transform(data['行业门类'])
    data['行业中类'] = str2int.fit_transform(data['行业中类'])
    data['街道乡镇'] = str2int.fit_transform(data['街道乡镇'])
    
    # 纳税人名称
    data['纳税人名称_count'] = data.groupby('纳税人名称')['纳税人名称'].transform('count')
    
    return data

In [124]:
def get_invoice(data):
    invoice = pd.DataFrame()

    for date in num_col_date:
        temp_col = ['主键:纳税人识别号','target'] + [date+i for i in num_col_type]
        temp_df = data[temp_col].copy()
        temp_df.columns = ['主键:纳税人识别号','target'] + [i for i in num_col_type]
        
        temp_df['date'] = int(date.replace('Q','0'))       
        invoice = pd.concat([invoice, temp_df], axis=0)

    invoice.sort_values(['主键:纳税人识别号','date'], inplace=True)
    invoice = invoice.reset_index(drop=True)
    
    # 提取一些特征
    invoice['have_na'] = invoice.isna().sum(axis=1)
    invoice['active'] = (np.sum(invoice[num_col_type], axis=1) > 0.1) * 1
    invoice['进项税额'] = (invoice['进项税率'] != 0.03) * invoice['进项税率'] * invoice['进项金额']
    invoice['预期入库税额'] = invoice['开票税额'] - invoice['进项税额']
    invoice['mismatch'] = (np.abs(invoice['入库税额'] -  invoice['预期入库税额']) > 0.5) * 1
    invoice['开票占比'] = invoice['开票数'] / invoice['发票核定数量']
    invoice['顶额率'] = invoice['开票金额'] / invoice['开票数']
    return invoice

In [125]:
def invoice_extract(data, invoice):
    # 活跃日期
    join = invoice.groupby('主键:纳税人识别号')['active'].sum().rename("active_count")
    data = data.join(join, on='主键:纳税人识别号')
    join = invoice.query('active==1').groupby(['主键:纳税人识别号'])['date'].first().rename("active_first_date")
    data = data.join(join, '主键:纳税人识别号')
    join = invoice.query('active==1').groupby(['主键:纳税人识别号'])['date'].last().rename("active_last_date")
    data = data.join(join, '主键:纳税人识别号')
    
    # 
    join = invoice.groupby('主键:纳税人识别号')['mismatch'].sum().rename('mismatch_sum')
    data = data.join(join, on='主键:纳税人识别号')
    print( data.query('mismatch_sum!=0 and target==1').shape[0] / data.query('mismatch_sum!=0').shape[0])
    # 
    
    aggs = {i:['std','max','min','mean','sum'] for i in num_col_type}
    join = invoice.query('active==1').groupby(['主键:纳税人识别号']).agg(aggs)
    join = join.fillna(0)
    
    cols = join.columns.to_list()
    auto_features = [i+'_'+j for i,j in cols]
    join.columns = auto_features
    data = data.join(join, on='主键:纳税人识别号', how='left')
    return data, auto_features

In [126]:
train = main_extract(train)
invoice = get_invoice(train)
num_col_type += ['进项税额','预期入库税额', '开票占比','顶额率']
train, auto_features = invoice_extract(train, invoice)

0.08517737969792764


In [127]:
train.shape

(16000, 197)

In [128]:
train1 = train.iloc[:train_len, :]
testb = train.iloc[-testb_len:, :]

In [141]:
def get_u_score(y_true, y_pred):
    temp = y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred)
    score = 1 - temp.sum() / np.sum(y_true) # len(y_true)
    return 'cross_entropy', score, False


def xgb_score(y_pred, dtrain):
    y_true = dtrain.get_label()
    temp = y_true*np.log(y_pred) + (1-y_true)*np.log(1-y_pred)
    score = -1 * temp.sum() / np.sum(y_true) # len(y_true)  
    return 'cross_entropy', score


class XGBWrapper(object): 
    def __init__(self, model=None):
        self.base_params = {
            'booster':'gbtree',
            'nthread':-1,
            
            #-------Booster Parameters
            'tree_method': "exact", #Choices: {'auto', 'exact', 'approx', 'hist', 'gpu_exact', 'gpu_hist'}
            'eta': 0.02,            
            'gamma':0,           
            'max_depth': 6, 
            'max_leaf_nodes':200,
            'max_delta_step':0,
            'subsample': 0.7,       
            'colsample_bytree': 0.8,  
            'colsample_bylevel':0.8, 
            'lambda':0.5,
            'alpha':0.5,
            'scale_pos_weight':1,   # 1 by default
            
            'objective': 'binary:logistic',
            'eval_metric': 'logloss',  # error
            'random_state': 88,  
        }
        if model != None:
            self.model = model

    def fit(self, X_train, y_train, X_valid=None, y_valid=None, cols_to_drop=[], params=None):
        cols = [col for col in X_train.columns if col not in cols_to_drop]
        self.cols_to_drop = cols_to_drop
        X_train = X_train[cols].copy()
        dtrain = xgb.DMatrix(X_train, y_train)
        watchlist = [(dtrain, 'train')]
        
        if X_valid is not None:
            X_valid = X_valid[cols].copy()
            dvalid = xgb.DMatrix(X_valid, y_valid)
            watchlist.append((dvalid, 'valid'))
        
        if params is not None:
            self.base_params.update(params)
        
        self.model = xgb.train(self.base_params, dtrain, num_boost_round=250, evals=watchlist, maximize=False,
                      verbose_eval=50, early_stopping_rounds=50,) #feval=xgb_score)
        
        return self.model
    
    def get_importance(self):
        assert(self.model is not None)
        
        df_imp = pd.DataFrame({'features': list(self.model.get_score().keys()), 
                               'importance': list(self.model.get_score().values())})
        df_imp = df_imp.sort_values(['importance'], ascending=False)
        return df_imp
    
    def predict(self, X_test):
        assert(self.model is not None)
        cols = [col for col in X_test.columns if col not in self.cols_to_drop]
        X_test = X_test[cols]
        dtest = xgb.DMatrix(X_test)
        return self.model.predict(dtest, ntree_limit=self.model.best_ntree_limit)


class KFold_tree():
    def __init__(self, model_wrapper):
        self.models = []
        self.model_wrapper = model_wrapper
    
    def fit(self, train, label, cols_to_drop, params=None, k=5):
        cols = [col for col in train.columns if col not in cols_to_drop]
        self.cols_to_drop = cols_to_drop
        train = train[cols]
        
        skfold = StratifiedKFold(n_splits=k, random_state=50)
        n_target = 1
        self.oof = np.zeros(len(train))
        
        for fold_n, (train_index, valid_index) in enumerate(skfold.split(train, label)):
            print(fold_n+1, 'fold start')
            X_train, X_valid = train.iloc[train_index], train.iloc[valid_index]
            y_train, y_valid = label.iloc[train_index], label.iloc[valid_index]
            model = self.model_wrapper.fit(X_train, y_train, X_valid, y_valid, params=params)
            self.models.append(model)
            self.oof[valid_index] = self.model_wrapper.predict(X_valid)
        
        self.score = get_u_score(label, self.oof)
        print('score of oof:', self.score)
    
    def get_importance(self):
        assert(len(self.models)>1)
        imp = pd.DataFrame()
        for model in self.models:
            self.model_wrapper.model = model
            single_imp = self.model_wrapper.get_importance()
            imp = imp.append(single_imp)
        return imp.groupby('features')['importance'].sum().reset_index().sort_values(['importance'], ascending=False)
    
    
    def predict(self, X_test):
        assert(len(self.models)>1)
        cols = [col for col in X_test.columns if col not in self.cols_to_drop]
        X_test = X_test[cols]
        self.model_wrapper.model = self.models[0]
        pred = self.model_wrapper.predict(X_test)
        for model in self.models[1:]:
            self.model_wrapper.model = model
            pred += self.model_wrapper.predict(X_test)
        
        return pred / len(self.models)


wrapper = XGBWrapper()
models = KFold_tree(wrapper)

In [142]:
drop_cols = ['财务人员手机号','财务人员姓名','登记开业日期','法定代表人姓名','法人手机号','经营范围','纳税人名称','生产经营地址','注册地址',
             '主键:纳税人识别号'] + ['target','preds','pred']

In [143]:
wrapper.fit(train1, train1['target'], cols_to_drop=drop_cols)

[0]	train-logloss:0.673566
Will train until train-logloss hasn't improved in 50 rounds.
[50]	train-logloss:0.203435
[100]	train-logloss:0.075033
[150]	train-logloss:0.031226
[200]	train-logloss:0.015103
[250]	train-logloss:0.00887
[299]	train-logloss:0.006325


In [144]:

result = pd.DataFrame()
result['zjnsrsbh'] = test['主键:纳税人识别号']
result['Probability'] = wrapper.predict(testb)
get_score.post_user_id('13')
get_score.post_verify_data(result)

User-id:13
程序开始运行时间为：2019-12-22 15:02:56.427928
程序结束运行时间为：2019-12-22 15:02:56.431330
程序运行时间（去除打分耗时）为：0.003402
程序的准确率为：93.23551316
最终得分为：93.91196185


In [37]:
# from ustciscrLab_A import get_score 

# result = pd.DataFrame()
# result['zjnsrsbh'] = test['主键:纳税人识别号']
# result['Probability'] = models.predict(testb)
# shit.post_user_id('13')
# shit.post_verify_data(result)